In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s52.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
52-00970,0,0,8,8,8,66.0,74,90.0,77,124.4,...,0,0,0,0,0,0,27,24,1,1
52-03198,0,0,8,8,8,66.0,74,70.0,77,124.4,...,0,0,0,0,0,0,26,24,1,1
52-05045,0,0,8,8,8,77.0,74,20.0,77,124.4,...,0,0,0,0,0,0,21,37,1,1
52-01722,0,0,8,8,8,99.0,74,0.0,77,124.4,...,0,0,0,0,0,0,24,28,1,1
52-00989,0,0,8,8,8,77.0,74,60.0,1,124.4,...,0,0,0,0,0,0,34,38,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-03765,0,0,8,8,8,77.0,74,0.0,77,124.4,...,0,0,0,0,1,0,36,24,1,1
52-00172,0,0,8,8,8,77.0,74,50.0,1,124.4,...,0,0,0,0,0,0,41,21,2,1
52-04394,0,0,8,8,8,99.0,74,0.0,77,124.4,...,0,0,0,0,1,0,23,24,1,1


In [3]:
X.shape

(6020, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(4214, 195)
(6020, 195)
(6020,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(646, 195)
(646,)
[323 323]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1217  450]
 [  16  123]]
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      1667
           1       0.21      0.88      0.35       139

    accuracy                           0.74      1806
   macro avg       0.60      0.81      0.59      1806
weighted avg       0.93      0.74      0.80      1806

ROC AUC prob1: 0.8478074169338794
MCC: 0.3521708333566601
F2: 0.5447298494242693
PR_AUC: 0.2858238860040616
CPU times: user 268 ms, sys: 0 ns, total: 26

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1239  428]
 [  17  122]]
              precision    recall  f1-score   support

           0       0.99      0.74      0.85      1667
           1       0.22      0.88      0.35       139

    accuracy                           0.75      1806
   macro avg       0.60      0.81      0.60      1806
weighted avg       0.93      0.75      0.81      1806

ROC AUC prob1: 0.8582233193649039
MCC: 0.3596293280150705
F2: 0.5515370705244123
PR_AUC: 0.30447775875825994
CPU times: user 1.13 s, sys: 5.49 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 17:24:17,700] A new study created in memory with name: no-name-cbb9348f-72d4-4886-b7a8-f915172baa87
[I 2021-12-10 17:24:18,307] Trial 0 finished with value: 0.36147334514803375 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.36147334514803375.


MCC: [0.3446576100619373, 0.3701820554869729, 0.36037798214316813, 0.3706757329000567]
ROC: [0.8466499843603379, 0.8573913043478261, 0.8487934758322783, 0.8640860455301507]
PR_AUC: [0.23900539023781747, 0.2903254718299719, 0.2770902795969193, 0.24345882511315797]


[I 2021-12-10 17:24:19,633] Trial 1 finished with value: 0.3626606208438515 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 1 with value: 0.3626606208438515.


MCC: [0.3504470831860561, 0.3613459385912328, 0.3728695450004163, 0.36597991659770074]
ROC: [0.8288645605254926, 0.8600250234594933, 0.8545592214691791, 0.8633412775253841]
PR_AUC: [0.2105331102698293, 0.324399737600382, 0.22871758588525656, 0.25365058344000596]


[I 2021-12-10 17:24:21,012] Trial 2 finished with value: 0.3698028368020847 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.3698028368020847.


...

MCC: [0.3619012441539235, 0.37445256163143475, 0.3715818227041864, 0.38344594077690497]
ROC: [0.8396746950265874, 0.8591429465123552, 0.847490131823937, 0.8718254263796827]
PR_AUC: [0.22101096137072074, 0.3270945421719344, 0.22666985092404934, 0.25272434672345745]


[I 2021-12-10 17:24:48,009] Trial 23 finished with value: 0.3683072337416541 and parameters: {'n_estimators': 36, 'subsample': 0.831722675533267, 'max_depth': 17, 'max_features': 20, 'learning_rate': 0.11087704486314144, 'min_samples_leaf': 31}. Best is trial 21 with value: 0.377204797975063.


MCC: [0.357037742736113, 0.36300536717503823, 0.37998852512795234, 0.3731972999275129]
ROC: [0.8314670003127934, 0.8582546137003441, 0.8426429333929149, 0.8678222983540627]
PR_AUC: [0.21711793044917913, 0.3208209540952861, 0.22326890878007005, 0.25603767236812713]


[I 2021-12-10 17:24:49,532] Trial 24 finished with value: 0.36624743801062654 and parameters: {'n_estimators': 99, 'subsample': 0.8799731267520511, 'max_depth': 14, 'max_features': 18, 'learning_rate': 0.08691890839385083, 'min_samples_leaf': 39}. Best is trial 21 with value: 0.377204797975063.


MCC: [0.34943594215992924, 0.37316791616035233, 0.36453976018166423, 0.37784613354056024]
ROC: [0.8411448232718174, 0.8598748827025336, 0.8472853206226261, 0.8729922295871503]
PR_AUC: [0.2213757834872554, 0.32754785988108426, 0.223647243667564, 0.2570076071817302]
CPU times: user 31.7 s, sys: 232 ms, total: 32 s
Wall time: 31.8 s


In [14]:
best = study.best_trial
best

FrozenTrial(number=21, values=[0.377204797975063], datetime_start=datetime.datetime(2021, 12, 10, 17, 24, 45, 370578), datetime_complete=datetime.datetime(2021, 12, 10, 17, 24, 46, 290307), params={'n_estimators': 58, 'subsample': 0.8592355833508208, 'max_depth': 16, 'max_features': 19, 'learning_rate': 0.09319382600103734, 'min_samples_leaf': 41}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=21, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[1201  466]
 [  10  129]]
              precision    recall  f1-score   support

           0       0.99      0.72      0.83      1667
           1       0.22      0.93      0.35       139

    accuracy                           0.74      1806
   macro avg       0.60      0.82      0.59      1806
weighted avg       0.93      0.74      0.80      1806

ROC AUC prob1: 0.8548980851311753
MCC: 0.3677592811415039
F2: 0.5603822762814944
PR_AUC: 0.24886704241749635
CPU times: user 80.7 ms, sys: 5 ms, total: 85.7 ms
Wall time: 83.2 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[1334  333]
 [  32  107]]
              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1667
           1       0.24      0.77      0.37       139

    accuracy                           0.80      1806
   macro avg       0.61      0.79      0.62      1806
weighted avg       0.92      0.80      0.84      1806

ROC AUC prob1: 0.856883299599073
MCC: 0.3539294885253634
F2: 0.5371485943775101
PR_AUC: 0.29758072889850945
CPU times: user 352 ms, sys: 9.07 ms, total: 361 ms
Wall time: 356 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delfetalpos    0.136056
Delmode        0.117158
Presentdel     0.081490
Lac_Min        0.068870
Admefface      0.048784
TrialLabor     0.046878
new_age        0.033182
BESTGA         0.032594
CS_FTP         0.032448
Analgesia      0.028164
Admreason      0.023778
Lac_None       0.022372
prelaborCD     0.018640
new_BMI        0.018010
spontlabor     0.017460
Augment        0.015708
Dilat_lst      0.012437
Episiotomy     0.011962
IUPC           0.011245
ROMmeth        0.010898
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode        0.397347
Delfetalpos    0.090754
CS_FTP         0.065424
Presentdel     0.055248
Admefface      0.037554
Admreason      0.033011
TrialLabor     0.028965
Lac_None       0.026507
new_age        0.022852
BESTGA         0.018979
Analgesia      0.018239
CS_UScar       0.017569
CS_Elect       0.017411
Lac_Min        0.014694
uscar          0.013476
DMControl      0.010380
Admcontract    0.010124
prelaborCD     0.009512
MthInd_Oxy     0.009113
AnteGDM        0.008989
dtype: float64

In [2]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [22]:
gb_params # 5 runs

{'n_estimators': 58,
 'subsample': 0.8592355833508208,
 'max_depth': 16,
 'max_features': 19,
 'learning_rate': 0.09319382600103734,
 'min_samples_leaf': 41,
 'random_state': 42}

In [3]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [25]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>